# Introduction to Data Science: Football Worldcup Data Analysis

In [ ]:
# Importing all the modules required
import numpy as np                  # numpy for linear algebra operations
import pandas as pd                 # pandas for datafile processing
import matplotlib.pyplot as plt     # matplotlib for data visualization
import seaborn as sns               # also for data visualation (graphs etc.)
%matplotlib inline
import plotly as py                 # for data visualisation (it supports a huge variety of plotable charts)
import cufflinks as cf              # it connects plotly with pandas so that we can create charts directly on data frames

In [ ]:
# Importing the datasets
players = pd.read_csv("/content/sample_data/WorldCupPlayers.csv")
matches = pd.read_csv("/content/sample_data/WorldCupMatches.csv")
world_cup = pd.read_csv("/content/sample_data/WorldCups.csv")

FileNotFoundError: ignored

In [ ]:
#First 5 rows of the players dataset
players.head()

# If we want, we can specify a number to get the first 'n' rows as well.
# Example, players.head(20)

In [ ]:
# First 5 rows of matches dataset
matches.head()

In [ ]:
# The last 5 entries in the matches dataset

matches.tail()

In [ ]:
# First 5 rows of worldcup dataset

world_cup.head()

## Data Cleaning

##### Our datasets contain various entries in which there is no data. So, we will remove (clean) those rows which contain NULL values.

In [ ]:
matches.dropna(subset=['Year'], inplace=True)

# It removes all entries where the 'Year' column had NULL values

In [ ]:
matches.tail()

# You can see the dataset after data cleaning

In [ ]:
# Getting the value count in a dataset (meaning how many time a specific value has been repeated)
matches['Home Team Name'].value_counts()

In [ ]:
matches['Home Team Goals'].value_counts()

In [ ]:
# Some Home Teams include unrequired characters.

names = matches[matches['Home Team Name'].str.contains('rn">')]['Home Team Name'].value_counts()

names

In [ ]:
# Creating a new list with wrong names of the Home Teams (they include the 'rn">' in the beginning)

wrong = list(names.index)

wrong

In [ ]:
# Removing the 'rn">' from the names. We do not require it.

correct = [name.split('>')[1] for name in wrong]

correct

In [ ]:
old_name = ['Germany FR', 'Maracan� - Est�dio Jornalista M�rio Filho', 'Estadio do Maracana'] # These teams also have some character malfunctioning in them.
new_name = ['Germany', 'Maracan Stadium', 'Maracan Stadium'] # These are the corrected names

In [ ]:
wrong = wrong + old_name #Adding oldname to the wrong names list
correct = correct + new_name #Adding newname to the correct names list

In [ ]:
wrong # Displaying the updated wrong team names list

In [ ]:
correct # Displaying the updated correct team names list

In [ ]:
# Replacing wrong values with corrected ones in players, matches and worldcup datasets

for index, wr in enumerate(wrong):
    world_cup = world_cup.replace(wrong[index], correct[index])

for index, wr in enumerate(wrong):
    matches = matches.replace(wrong[index], correct[index])

for index, wr in enumerate(wrong):
    players = players.replace(wrong[index], correct[index])

In [ ]:
# Checking if there are still any wrong entries listed in the dataset

names = matches[matches['Home Team Name'].str.contains('rn">')]['Home Team Name'].value_counts()
names

## Most Number of World Cup Winning Title

In [ ]:
# Counting the number of times each country has appeared as the 'Winner' in the worldcup dataset

winner = world_cup['Winner'].value_counts()

winner

In [ ]:
# Counting the number of times each country has appeared as the 'Runner Up' in the worldcup dataset

runnerup = world_cup['Runners-Up'].value_counts()

runnerup

In [ ]:
# Counting the number of times each country has appeared as the 'Third' in the worldcup dataset

third = world_cup['Third'].value_counts()

third

In [ ]:
# Creating a new table listing the all the above values

teams = pd.concat([winner, runnerup, third], axis=1)     # Concating the three values with the heading
teams.fillna(0, inplace=True)   # Replaces the NULL values with a 0
teams = teams.astype(int)       # Converts all dtypes to int

teams

In [ ]:
# Importing iplot from plotly and connecting it to offline mode

from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
teams.iplot(kind = 'bar', xTitle='Teams', yTitle='World Cup Winning Count', title='FIFA World Cup Winning Count')

## Number of Goal Per Countary

In [ ]:
matches.head(2)

In [ ]:
# Cleaning the NULL fields from Teams and Goals and storing output to new lists

home = matches[['Home Team Name', 'Home Team Goals']].dropna()
away = matches[['Away Team Name', 'Away Team Goals']].dropna()

In [ ]:
# Setting the columns as 'Countries' and 'Goals' for home and away teams

home.columns = ['Countries', 'Goals']
away.columns = home.columns

In [ ]:
# Creating a new list combining both values from home and away

goals = home.append(away, ignore_index = True)

In [ ]:
# Grouping the countries by total goals they have scored (as home teams + as away teams)

goals = goals.groupby('Countries').sum()
goals

In [ ]:
# Sorting the list in descending order (most to less goals)

goals = goals.sort_values(by = 'Goals', ascending=False)

goals

In [ ]:
# Plotting graph for the most number of goals by countries
# Note that we are only displaying the top 30 countries from the list

goals[:30].iplot(kind='bar', xTitle = 'Country', yTitle = 'Number of Goals', title = 'Countries Hits Number of Goals')

## Attendance, Number of Teams, Goals, and Matches per Cup

In [ ]:
# Removing the decimals from the attendance column

world_cup['Attendance'] = world_cup['Attendance'].str.replace(".", "")

In [ ]:
world_cup.head(10)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 7))
sns.despine(right = True)
g = sns.barplot(x = 'Year', y = 'QualifiedTeams', data = world_cup)
g.set_xticklabels(g.get_xticklabels(), rotation = 80)
g.set_title('Qualified Teams Per Year')

#======================

fig, ax = plt.subplots(figsize = (10,5))
sns.despine(right = True)
g = sns.barplot(x = 'Year', y = 'GoalsScored', data = world_cup)
g.set_xticklabels(g.get_xticklabels(), rotation = 80)
g.set_title('Goals Scored per Year')

#======================

fig, ax = plt.subplots(figsize = (10,5))
sns.despine(right = True)
g = sns.barplot(x = 'Year', y = 'MatchesPlayed', data = world_cup)
g.set_xticklabels(g.get_xticklabels(), rotation = 80)
g.set_title('Matches Plyed Scored by Teams Per Year')

# Goals Per Team Per World Cup

In [ ]:
# Creating a new list which stores the grouped Year and Home teams
home = matches.groupby(['Year', 'Home Team Name'])['Home Team Goals'].sum()

home

In [ ]:
# Similarly, creating a list for away teams (which also stores goals scored by grouping them by team and then adding the total)
away = matches.groupby(['Year', 'Away Team Name'])['Away Team Goals'].sum()
away

In [ ]:
# New list 'goals' which concats 'home' and 'away' teams
goals = pd.concat([home, away], axis=1)
# Replace all NULL values with 0
goals.fillna(0, inplace=True)
# Adding the home and away team goals
goals['Goals'] = goals['Home Team Goals'] + goals['Away Team Goals']
# Removing the Home Team Goals and Away Team Goals titles from list 'goals'
goals = goals.drop(labels = ['Home Team Goals', 'Away Team Goals'], axis = 1)


goals

In [ ]:
# Reset the list index to default(default starts from 0)
goals = goals.reset_index()

In [ ]:
# Adding column headers
goals.columns = ['Year', 'Country', 'Goals']

# Sort list to ascending (by year) and in descending (by goal)
goals = goals.sort_values(by = ['Year', 'Goals'], ascending = [True, False])

goals



In [ ]:
# Top 5 goals by each year
top5 = goals.groupby('Year').head()

top5.head(10)

In [ ]:
import plotly.graph_objects as go

In [ ]:
x, y = goals['Year'].values, goals['Goals'].values

In [ ]:
data = []

# Looping over the top5 list (with removed duplicates)
for team in top5['Country'].drop_duplicates().values:
    year = top5[top5['Country'] == team]['Year']
    goal = top5[top5['Country'] == team]['Goals']

#     Appending the Bargraphs to the end of data list
    data.append(go.Bar(x = year, y = goal, name = team))
# Cerating a layout with stack as bargraph
layout = go.Layout(barmode = 'stack', title = 'Top 5 Teams with Most Goals Scored', showlegend = True)
# Creating the bargraph figure (diagram) with 'data' as data list and 'layout' as layout list
fig = go.Figure(data = data, layout = layout)
fig.show()

# Matches With Highest Number Of Attendance

In [ ]:
# Converting the date to datetime dtype
matches['Datetime'] = pd.to_datetime(matches['Datetime'])

In [ ]:
# Stringifying the date format
# %d - date
# %b - abbrevated month name
# %y - year (YY format)

matches['Datetime'] = matches['Datetime'].apply(lambda x: x.strftime('%d %b, %y'))
matches['Datetime']

In [ ]:
# Sorting the top 10 matches
top10 = matches.sort_values(by = 'Attendance', ascending = False)[:10]
top10['vs'] = top10['Home Team Name'] + " vs " + top10['Away Team Name']

# Creating the plot figure with size (12, 10)
plt.figure(figsize = (12,10))

ax = sns.barplot(y = top10['vs'], x = top10['Attendance'])
sns.despine(right = True)

plt.ylabel('Match Teams')
plt.xlabel('Attendence')
plt.title('Matches with the Highest Number of Attendence')

for i, s in enumerate("Stadium: " + top10['Stadium'] +", Date: " + top10['Datetime']):
    ax.text(2000, i, s, fontsize = 12, color = 'white')
plt.show()


# Stadium with Highest Average Attendance

In [ ]:
matches['Year'] = matches['Year'].astype(int)

std = matches.groupby(['Stadium', 'City'])['Attendance'].mean().reset_index().sort_values(by = 'Attendance', ascending =False)

top10 = std[:10]

plt.figure(figsize = (12,9))
ax = sns.barplot(y = top10['Stadium'], x = top10['Attendance'])
sns.despine(right = True)

plt.ylabel('Stadium Names')
plt.xlabel('Attendance')
plt.title('Stadium with the heighest number of attendance')
for i, s in enumerate("City: " + top10['City']):
        ax.text(2000, i, s, fontsize = 12, color = 'white')

plt.show()

In [ ]:
# Most matches were played in which cities
matches['City'].value_counts()[:20].iplot(kind = 'bar')

# Which countries had won the cup ?

In [ ]:
# Creating 3 lists categorising Winner, Runners-Up and Third positions
gold = world_cup["Winner"]
silver = world_cup["Runners-Up"]
bronze = world_cup["Third"]

# Counting values from the lists (we have used from_dict() method as we are creating a list from dict type)
gold_count = pd.DataFrame.from_dict(gold.value_counts())
silver_count = pd.DataFrame.from_dict(silver.value_counts())
bronze_count = pd.DataFrame.from_dict(bronze.value_counts())

# Performing Outer join
podium_count = gold_count.join(silver_count, how='outer').join(bronze_count, how='outer')
# Filling NULL values with 0
podium_count = podium_count.fillna(0)
# Adding column titles
podium_count.columns = ['WINNER', 'SECOND', 'THIRD']
# Converting the dtype of list to int64
podium_count = podium_count.astype('int64')
# Sorting values in descending order
podium_count = podium_count.sort_values(by=['WINNER', 'SECOND', 'THIRD'], ascending=False)

# Plotting the bargraph
podium_count.plot(y=['WINNER', 'SECOND', 'THIRD'], kind="bar",
                  color =['gold','silver','brown'], figsize=(15, 6), fontsize=14,
                 width=0.8, align='center')
plt.xlabel('Countries')
plt.ylabel('Number of podium')
plt.title('Country-wise Podium Count')

In [ ]:
# Removing NULL values
home = matches[['Home Team Name', 'Home Team Goals']].dropna()
# Removing NULL values
away = matches[['Away Team Name', 'Away Team Goals']].dropna()

# Creating a new DataFrame with countires and goals
goal_per_country = pd.DataFrame(columns=['countries', 'goals'])
# Appending away to home and then assigning the resultant to goal_per_country dataframe
goal_per_country = goal_per_country.append(home.rename(index=str, columns={'Home Team Name': 'countries', 'Home Team Goals': 'goals'}))
goal_per_country = goal_per_country.append(away.rename(index=str, columns={'Away Team Name': 'countries', 'Away Team Goals': 'goals'}))

# Converting the dtype of goals
goal_per_country['goals'] = goal_per_country['goals'].astype('int64')

# Grouping the countries by country name, adding the goal values and then sorting them in descending order
goal_per_country = goal_per_country.groupby(['countries'])['goals'].sum().sort_values(ascending=False)

# Plotting the graph
goal_per_country[:10].plot(x=goal_per_country.index, y=goal_per_country.values, kind="bar", figsize=(12, 6), fontsize=14)
plt.xlabel('Countries')
plt.ylabel('Number of goals')
plt.title('Top 10 of Number of goals by country')

# Match outcome by home and away teams

In [ ]:
# Creating a function get_labels() which takes in a match as argument and returns if the Home Team has more goals or the Away Team had more goals.
# If both teams scored equally, it returns 'DRAW'
def get_labels(matches):
    if matches['Home Team Goals'] > matches['Away Team Goals']:
        return 'Home Team Win'
    if matches['Home Team Goals'] < matches['Away Team Goals']:
        return 'Away Team Win'
    return 'DRAW'

In [ ]:
# Creating a new column 'outcome' in matches dataframe and performing the get_labels() function
l = matches.apply(lambda x: get_labels(x), axis=1)

In [ ]:
# Matches dataframe after executing get_label() function
matches.head()

In [ ]:
# Counting the total result of matchs played
match_outcomes = matches['outcome'].value_counts()
match_outcomes

In [ ]:
# Plotting graph on match_outcomes
plt.figure(figsize = (6,6))

match_outcomes.plot.pie(autopct = "%1.0f%%", colors = sns.color_palette('winter_r'), shadow = True)

c = plt.Circle((0,0), 0.4, color =  'white')
plt.gca().add_artist(c)
plt.title('Match Outcomes by Home and Away Teams')
plt.show()